# NLP Project

## Load Data

### Scraped CDP Scores
See [./download_csp_scores.py](./download_cdp_scores.py) for the CDP scores scraping code

In [2]:
import pandas as pd
import numpy as np
import platform
from transformers import pipeline
import tensorflow as tf
import torch
from flax import nnx
pd.options.display.max_colwidth = 100

/Users/jacksonmiskill/class/f24/nlp/project/venv-3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-29 18:16:14.938187: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
p = platform.platform()

esg_documents_df = pd.read_csv('./dax_esg_media_dataset/esg_documents_for_dax_companies.csv', sep='|')
cdp_scores_df = pd.read_csv('./cdp_scores.csv')
esg_documents_df = esg_documents_df.drop("Unnamed: 0", axis=1)



In [10]:
# merge CDP score with the esg data so that we have a score for each document
merged_df = pd.merge(esg_documents_df, cdp_scores_df, how='left', left_on='symbol', right_on='Ticker')

merged_df.dropna(subset=['CDP Score'], inplace=True) # drop rows without a CDP score

# print(merged_df[['symbol', 'CDP Score']])
# print(len(merged_df))
s = merged_df.head(1)['content'].item() # this is how to get the text alone

In [ ]:
# big companies that I know
# Adidas, Volkswagen, Porsche, Siemens (AG and Energy), Merck, Airbus, Puma, Allianz, Brenntag


## Preprocessing

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    lowercase=True,
    stop_words='english',
    token_pattern=r'\b\w+\b' # Only keep word tokens
)

bow_matrix = vectorizer.fit_transform(merged_df['content'].fillna(''))
y = merged_df['CDP Score']
# can use the bow_matrix in the next step for building a model

In [ ]:
# https://www.geeksforgeeks.org/text-classification-using-logistic-regression/#
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# runs out of iterations because our training data is huge (several entire documents per company)
X_train, X_test, y_train, y_test = train_test_split(bow_matrix, y, test_size=0.25, random_state=1)


model = LogisticRegression(random_state=1)
model.fit(X_train, y_train)



/Users/jacksonmiskill/class/f24/nlp/project/venv-3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=1)

In [18]:
message = "our carbon production has increased by 150% this past year"
vect_message = vectorizer.transform([message])
pred = model.predict(vect_message)
pred

array(['A-'], dtype=object)

## Model Fitting

In [ ]:
# TODO here we should try and build a model that correlates the preprocessed
# data to the column of our choice in the sp500_risk_ratings dataset


CountVectorizer(stop_words='english', token_pattern='\\b\\w+\\b')

In [ ]:
# this is the FinBERT model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# examples of text we can feed the model in order for it to classify it
# these examples are from the DAX dataset we are looking at
positive_example = "we have reduced our carbon emissions by 12.7% (vs. 2018)"
neutral_example = "We are proud of this result"
negative_example = "We have increased our carbon emissions by 12.7% (vs. 2018), which is a very bad thing"
false_positive = "we have increased our carbon emissions by 12.7% (vs. 2018)"

inputs = tokenizer(false_positive, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'positive'

In [32]:
# climateBERT model from https://huggingface.co/climatebert/environmental-claims?library=transformers
# environmental claims model - I wasn't sure which one to pick from their suite of models, this one semantically made the most sense to me
# there were a bunch of other ones

from transformers import AutoTokenizer, AutoModelForSequenceClassification

climate_tokenizer = AutoTokenizer.from_pretrained("climatebert/environmental-claims")
climate_model = AutoModelForSequenceClassification.from_pretrained("climatebert/environmental-claims")


inputs = climate_tokenizer(false_positive, return_tensors="pt")

with torch.no_grad():
    logits = climate_model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'negative'

as we can see here, the second model correctly predicts the false positive whereas the first model does not

## Model Analysis

In [ ]:
# TODO check how well our model did here

# Data Sources
- https://www.kaggle.com/datasets/pritish509/s-and-p-500-esg-risk-ratings
- https://www.kaggle.com/datasets/equintel/dax-esg-media-dataset

# References

# Scratch Work

In [4]:
import pandas as pd
pd.read_csv("~/class/f24/nlp/project/sustain-nlp/sp500_esg_risk_ratings/sp500_esg_risk_ratings.csv")

,Symbol,Name,Address,Sector,Industry,Full Time Employees,Description,Total ESG Risk score,Environment Risk Score,Governance Risk Score,Social Risk Score,Controversy Level,Controversy Score,ESG Risk Percentile,ESG Risk Level
0,ENPH,"Enphase Energy, Inc.","47281 Bayside Parkway\nFremont, CA 94538\nUnit...",Technology,Solar,"3,157","Enphase Energy, Inc., together with its subsid...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EMN,Eastman Chemical Company,"200 South Wilcox Drive\nKingsport, TN 37662\nU...",Basic Materials,Specialty Chemicals,"14,000",Eastman Chemical Company operates as a special...,25.3,12.8,6.6,5.8,Moderate Controversy Level,2.0,50th percentile,Medium
2,DPZ,Domino's Pizza Inc.,"30 Frank Lloyd Wright Drive\nAnn Arbor, MI 481...",Consumer Cyclical,Restaurants,"6,500","Domino's Pizza, Inc., through its subsidiaries...",29.2,10.6,6.3,12.2,Moderate Controversy Level,2.0,66th percentile,Medium
3,DAY,"Dayforce, Inc.","3311 East Old Shakopee Road\nMinneapolis, MN 5...",Technology,Software - Application,"9,084","Dayforce Inc., together with its subsidiaries,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DVA,Davita Inc.,"2000 16th Street\nDenver, CO 80202\nUnited States",Healthcare,Medical Care Facilities,"70,000",DaVita Inc. provides kidney dialysis services ...,22.6,0.1,8.4,14.1,Moderate Controversy Level,2.0,38th percentile,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,ACN,Accenture Plc,1 Grand Canal Square\nGrand Canal Harbour\nDub...,Technology,Information Technology Services,"742,000","Accenture plc, a professional services company...",9.8,0.8,4.4,4.6,Moderate Controversy Level,2.0,3rd percentile,Negligible
499,ABBV,Abbvie Inc.,"1 North Waukegan Road\nNorth Chicago, IL 60064...",Healthcare,Drug Manufacturers - General,"50,000","AbbVie Inc. discovers, develops, manufactures,...",29.9,2.4,10.4,17.2,Significant Controversy Level,3.0,69th percentile,Medium
500,ABT,Abbott Laboratories,100 Abbott Park Road\nAbbott Park\nNorth Chica...,Healthcare,Medical Devices,"114,000","Abbott Laboratories, together with its subsidi...",24.8,2.3,8.3,14.2,Significant Controversy Level,3.0,48th percentile,Medium
501,AOS,A.O. Smith Corporation,11270 West Park Place\nSuite 170 PO Box 245008...,Industrials,Specialty Industrial Machinery,"12,000",A. O. Smith Corporation manufactures and marke...,25.4,7.2,6.4,11.9,Low Controversy Level,1.0,51st percentile,Medium


In [ ]:
# if you want to find text data from S&P companies - this is the code to load it
# source: https://www.kaggle.com/datasets/jaidityachopra/esg-sustainability-reports-of-s-and-p-500-companies 
# pd.read_csv("~/class/f24/nlp/project/extra_data/preprocessed_content.csv").head(1)['preprocessed_content']

0    style guide colour colour use imagecolour prof...
Name: preprocessed_content, dtype: object